# Analyze Model Weights

In [1]:
import pandas as pd
import numpy as np

%load_ext autoreload
%autoreload 2

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_theme(
    context="paper", 
    style="whitegrid", 
    font_scale=1.2,
    rc={'figure.figsize': (20, 20), 'figure.dpi': 300}
)

## Load Weights

In [18]:
from common.data import get_model_weight

population = 'adhd'
measure = 'WISC_PSI'
age_group = 'all'

pls_weights = get_model_weight('pls', population, measure, age_group)
ridge_weights = get_model_weight('ridge', population, measure, age_group)

print(pls_weights.shape, ridge_weights.shape)

(34716,) (34716,)


In [9]:
from os.path import join
from common.paths import RIDGE_WEIGHTS

weight_pop = 'adhd'
weight_tar = 'WISC_FSIQ'
weight_age = 'all'
s1_f = f'ridge_{weight_pop}_{weight_tar}_{weight_age}_set_2.npy'
s2_f = f'ridge_healthy_{weight_tar}_{weight_age}_set_1.npy'

s1 = np.load(join(RIDGE_WEIGHTS, s1_f))
s2 = np.load(join(RIDGE_WEIGHTS, s2_f))

## Intraclass Correlation (ICC)

In [10]:
%%time
import pingouin as pg

# coefs = np.array([pls_weights, ridge_weights])
coefs = np.array([s1, s2])
# print(f'{population}_{measure}_{age_group}', coefs.shape)

icc_data = pd.DataFrame(coefs).melt(var_name='connection', value_name='weight', ignore_index=False)
icc_data['cv_run_num'] = icc_data.index
icc = pg.intraclass_corr(data=icc_data, targets='connection', raters='cv_run_num', ratings='weight').round(3)
icc.set_index("Type")

CPU times: user 29.7 s, sys: 196 ms, total: 29.9 s
Wall time: 29.9 s


,Description,ICC,F,df1,df2,pval,CI95%
Type,,,,,,,
ICC1,Single raters absolute,0.121,1.276,34715,34716,0.0,"[0.11, 0.13]"
ICC2,Single random raters,0.121,1.277,34715,34715,0.0,"[0.11, 0.13]"
ICC3,Single fixed raters,0.122,1.277,34715,34715,0.0,"[0.11, 0.13]"
ICC1k,Average raters absolute,0.216,1.276,34715,34716,0.0,"[0.2, 0.23]"
ICC2k,Average random raters,0.217,1.277,34715,34715,0.0,"[0.2, 0.23]"
ICC3k,Average fixed raters,0.217,1.277,34715,34715,0.0,"[0.2, 0.23]"


## Plot Weights

In [ ]:
from common.plotting import create_power_fc_matrix, plot_connections
from mne.viz import circular_layout, plot_connectivity_circle

### Nilearn

In [ ]:
plot_connections(selected_features, -0.035, 0.035, True)

### MNE

In [ ]:
from operator import itemgetter
from common.paths import POWER

power_labels = pd.read_csv(POWER, index_col='ROI')
node_names = [str(node_num) for node_num in range(1, 265)]
node_labels = list(zip(node_names, power_labels['Assignment']))
node_labels.sort(key=itemgetter(1))
node_order = [node[0] for node in node_labels]

num_nodes_per_system = power_labels['Assignment'].value_counts().sort_index()
group_boundaries = np.array(np.cumsum(num_nodes_per_system.values)[:-1])

node_colors = power_labels['Color'].values.tolist()

node_angles = circular_layout(node_names, node_order, group_boundaries=group_boundaries,
                              group_sep=0)

In [ ]:
matrix = create_power_fc_matrix(selected_features)

# plot_connectivity_circle(matrix, node_names, n_lines=300, node_angles=node_angles, 
#                          fontsize_names=4, padding=0, colormap='bwr', linewidth=1, show=True,
#                          node_colors=node_colors)
node_names = ["" for node_num in range(1, 265)]
fig, _ = plot_connectivity_circle(matrix, node_names, n_lines=300, node_angles=node_angles, 
                         fontsize_names=4, padding=0, colormap='bwr', linewidth=1, show=True,
                         node_colors=node_colors, facecolor=(0, 0, 0, 0), node_edgecolor=(0, 0, 0, 0),
                         colorbar=True, node_width=0.9)
fig.savefig('/home/bpho/Documents/MSc_Research-Project/images/demo.png', transparent=True)

## Analyze Nodes

In [ ]:
from nilearn import plotting, datasets
from common.calculation import get_k_argmax
from common.plotting import create_power_fc_matrix, convert_fc_to_node_strength

power = datasets.fetch_coords_power_2011()
coords = np.vstack((power.rois['x'], power.rois['y'], power.rois['z'])).T
matrix = create_power_fc_matrix(pls_coef)

# clip connectivity matrix to preserve positive and negative edges
positive_edges = np.clip(matrix, 0, matrix.max())
negative_edges = np.clip(matrix, matrix.min(), 0)

# calculate strength for positive edges
node_strength_positive = convert_fc_to_node_strength(positive_edges)

# calculate strength for negative edges
node_strength_negative = convert_fc_to_node_strength(negative_edges)

print(node_strength_positive.shape, node_strength_negative.shape)
top_pos_nodes = get_k_argmax(node_strength_positive, 10)
top_neg_nodes = get_k_argmax(node_strength_negative, 10)

print(coords[top_pos_nodes], node_strength_positive[top_pos_nodes])
print(coords[top_neg_nodes], node_strength_negative[top_neg_nodes])

plotting.plot_markers(node_strength_positive[top_pos_nodes], coords[top_pos_nodes],
                      node_vmin=0, node_vmax=1, node_cmap="YlOrRd")
plotting.plot_markers(node_strength_negative[top_neg_nodes], coords[top_neg_nodes],
                      node_vmin=0, node_vmax=1, node_cmap="PuBu")